## 3. Mapping, sorting, and calling trimmed data

Using [NextGenMap](https://github.com/Cibiv/NextGenMap) for mapping and [samtools](https://github.com/samtools/samtools) for sorting. Alternatives are [minimap2](https://github.com/lh3/minimap2) and [Sambamba](https://github.com/biod/sambamba). Even a quite powerful personal desktop may freeze up or not start running MiniMap2 due to its memory requirements. NextGenMap seems to play better but when dealing with a HPC cluster, either should work fine.

After this, we will use [FreeBayes](https://github.com/ekg/freebayes) for variant detection on the mapped and sorted files. Working with BAM files is a bit faster than working with CRAM but CRAM is more space efficient. As such, all code utilizes BAM but at the end, data will be converted to CRAM for data access after publication.

In [9]:
##!conda install -c bioconda samtools minimap2 sambamba
##!conda install -c anaconda openssl=1.0.2
##!git clone https://github.com/lh3/bwa.git
##!cd bwa; make
##!cd ..
import os

In [1]:
%time !gunzip < /moto/eaton/projects/macaques/refpapio/refpapio.fna.gz > /moto/eaton/projects/macaques/refpapio/refpapio.fa

CPU times: user 125 ms, sys: 46.7 ms, total: 172 ms
Wall time: 19.8 s


In [4]:
%time !samtools faidx /moto/eaton/projects/macaques/refpapio/refpapio.fa

samtools: error while loading shared libraries: libcrypto.so.1.0.0: cannot open shared object file: No such file or directory
CPU times: user 522 µs, sys: 6.08 ms, total: 6.6 ms
Wall time: 133 ms


In [ ]:
%time !./bwa index /moto/eaton/projects/macaques/refpapio/refpapio.fa

### A) Mapping and sorting northern _Macaca mulatta_:

In [1]:
##creating a folder to put the BAM files into 
!mkdir /moto/eaton/projects/macaques/mapped/

In [2]:
test=['SRR5628058', 'fasno', 'SRR2981139', 'SRR1024051', 'silenus', 'sylvanus']

In [ ]:
##6 hrs 61gb BAM fastq1 24g fastq2 31g
%time !minimap2 -ax sr -t 20 -K 6g -2 /moto/eaton/projects/macaques/refpapio/refpapio.fa \
            /moto/eaton/projects/macaques/filteredfastq/SRR5628058.filtered_1.fastq.gz \
            /moto/eaton/projects/macaques/filteredfastq/SRR5628058.filtered_2.fastq.gz \
            | samtools view -b -> /moto/eaton/projects/macaques/mapped/SRR5628058.raw.bam

In [ ]:
## fastq1 34g fastq2 38g
%time !ngm -t 20 -r /moto/eaton/projects/macaques/refpapio/refpapio.fna.gz \
    -1 /moto/eaton/projects/macaques/filteredfastq/sylvanus.filtered_1.fastq.gz \
    -2 /moto/eaton/projects/macaques/filteredfastq/sylvanus.filtered_2.fastq.gz \
    | samtools view -b -@ 3 -> /moto/eaton/projects/macaques/mapped/ngmsylvanus.raw.bam

In [ ]:
%time !ngm -t 20 -r /moto/eaton/projects/macaques/refpapio/refpapio.fna.gz \
    -1 /moto/eaton/projects/macaques/filteredfastq/SRR2981139.filtered_1.fastq.gz \
    -2 /moto/eaton/projects/macaques/filteredfastq/SRR2981139.filtered_2.fastq.gz \
    | samtools view -b -@ 3 -> /moto/eaton/projects/macaques/mapped/ngmSRR2981139.raw.bam

In [ ]:
%time !ngm -t 20 -r /moto/eaton/projects/macaques/refpapio/refpapio.fna.gz \
    -1 /moto/eaton/projects/macaques/filteredfastq/silenus.filtered_1.fastq.gz \
    -2 /moto/eaton/projects/macaques/filteredfastq/silenus.filtered_2.fastq.gz \
    | samtools view -b -@ 3 -> /moto/eaton/projects/macaques/mapped/ngmsilenus.raw.bam

In [ ]:
%time !ngm -t 20 -r /moto/eaton/projects/macaques/refpapio/refpapio.fna.gz \
    -1 /moto/eaton/projects/macaques/filteredfastq/fasno.filtered_1.fastq.gz \
    -2 /moto/eaton/projects/macaques/filteredfastq/fasno.filtered_2.fastq.gz \
    | samtools view -b -@ 3 -> /moto/eaton/projects/macaques/mapped/ngmfasno.raw.bam

In [ ]:
%time !ngm -t 20 -r /moto/eaton/projects/macaques/refpapio/refpapio.fna.gz \
    -1 /moto/eaton/projects/macaques/filteredfastq/SRR1024051.filtered_1.fastq.gz \
    -2 /moto/eaton/projects/macaques/filteredfastq/SRR1024051.filtered_2.fastq.gz \
    | samtools view -b -@ 3 -> /moto/eaton/projects/macaques/mapped/ngmSRR1024051.raw.bam

In [ ]:
##mapping
for i in test:
    cmd='minimap2 -ax sr -t 19 /moto/eaton/projects/macaques/refpapio/refpapio.fa \
            /moto/eaton/projects/macaques/filteredfastq/'+i+'.filtered_1.fastq.gz \
            /moto/eaton/projects/macaques/filteredfastq/'+i+'.filtered_2.fastq.gz \
            | samtools view -b -> /moto/eaton/projects/macaques/mapped/'+i+'.raw.bam'
    os.system(cmd)

In [4]:
test=['ngmfasno', 'ngmSRR2981139', 'ngmSRR1024051', 'ngmsilenus', 'ngmsylvanus']

In [ ]:
##sorting mapped data
for i in test:
    cmd='sambamba sort -t 20 --tmpdir=/moto/eaton/projects/macaques/tmp/ \
            /moto/eaton/projects/macaques/mapped/'+i+'.raw.bam'
    os.system(cmd)

In [2]:
test=['ngmfasno']

In [ ]:
##marking likely PCR duplicates
for i in test:
    cmd='sambamba markdup -t 20 -p --overflow-list-size 6000000 --hash-table-size 6000000 \
            --tmpdir=/moto/eaton/projects/macaques/tmp/ \
            /moto/eaton/projects/macaques/mapped/'+i+'.raw.sorted.bam \
            /moto/eaton/projects/macaques/mapped/'+i+'.mark.bam'
    os.system(cmd)

In [ ]:
test=['ngmSRR2981139', 'ngmSRR1024051', 'ngmsilenus', 'ngmsylvanus']

In [ ]:
for i in test:
    cmd='sambamba markdup -t 20 -p --overflow-list-size 6000000 --hash-table-size 6000000 \
            --tmpdir=/moto/eaton/projects/macaques/tmp/ \
            /moto/eaton/projects/macaques/mapped/'+i+'.raw.sorted.bam \
            /moto/eaton/projects/macaques/mapped/'+i+'.mark.bam'
    os.system(cmd)

### B) Calling Variants with FreeBayes

In [ ]:
test=['SRR5628058', 'fasno', 'SRR2981139', 'silenus', 'sylvanus']

In [ ]:
##!conda install -c bioconda freebayes

In [2]:
!mkdir /moto/eaton/projects/macaques/calls/

In [ ]:
%%bash
freebayes-parallel <(fasta_generate_regions.py /moto/eaton/projects/macaques/refpapio/refpapio.fa 100000) 20 \
    -f /moto/eaton/projects/macaques/refpapio/refpapio.fa \
    /moto/eaton/projects/macaques/mapped/SRR5628058.mark.bam \
    /moto/eaton/projects/macaques/mapped/ngmfasno.mark.bam \
    /moto/eaton/projects/macaques/mapped/ngmSRR2981139.mark.bam \
    /moto/eaton/projects/macaques/mapped/ngmSRR1024051.mark.bam \
    /moto/eaton/projects/macaques/mapped/ngmsilenus.mark.bam \
    /moto/eaton/projects/macaques/mapped/ngmsylvanus.mark.bam \
    >/moto/eaton/projects/macaques/calls/test.raw1.vcf    

In [17]:
##testing on one chromosome amongst all the samples start time = 
%time !freebayes -f /moto/eaton/projects/macaques/refpapio/refpapio.fa \
    -r NC_018152.2 \
    /moto/eaton/projects/macaques/mapped/SRR5628058.mark.bam \
    /moto/eaton/projects/macaques/mapped/ngmfasno.mark.bam \
    /moto/eaton/projects/macaques/mapped/ngmSRR2981139.mark.bam \
    /moto/eaton/projects/macaques/mapped/ngmSRR1024051.mark.bam \
    /moto/eaton/projects/macaques/mapped/ngmsilenus.mark.bam \
    /moto/eaton/projects/macaques/mapped/ngmsylvanus.mark.bam \
    > /moto/eaton/projects/macaques/calls/test.raw.vcf

^C
CPU times: user 1.92 s, sys: 549 ms, total: 2.47 s
Wall time: 5min 19s


In [ ]:
freebayes-parallel <(fasta_generate_regions.py ref.fa.fai 100000) 36 \
    -f ref.fa aln.bam >var.vcf

Finding what are the names of the scaffolds that reads were mapped to:

In [6]:
!samtools idxstats /moto/eaton/projects/macaques/mapped/ngmfasno.mark.bam \
    | cut -f 1 | head -22

NC_018152.2
NC_018153.2
NC_018154.2
NC_018155.2
NC_018156.2
NC_018157.2
NC_018158.2
NC_018159.2
NC_018160.2
NC_018161.2
NC_018162.2
NC_018163.2
NC_018164.2
NC_018165.2
NC_018166.2
NC_018167.2
NC_018168.2
NC_018169.2
NC_018170.2
NC_018171.2
NC_018172.2
NW_018761063.1
cut: write error: Broken pipe


Splitting by chromosome (NC_* is chromosome or mitochondria and NW_* is unassigned, so we just need the NC names):

In [7]:
test=['SRR5628058', 'ngmfasno', 'ngmSRR2981139', 'ngmSRR1024051', 'ngmsilenus', 'ngmsylvanus']

In [11]:
for i in test:
    cmd='samtools view -@7 \
            /moto/eaton/projects/macaques/mapped/'+i+'.mark.bam NC_018152.2 -b > \
            /moto/eaton/projects/macaques/mapped/'+i+'.NC_018152.2.bam'
    os.system(cmd)

In [13]:
for i in test:
    cmd='sambamba sort -t 20 --tmpdir=/moto/eaton/projects/macaques/tmp/ \
            /moto/eaton/projects/macaques/mapped/'+i+'.NC_018152.2.bam'
    os.system(cmd)

In [ ]:
for i in test:
    cmd='sambamba markdup -t 20 -p --overflow-list-size 6000000 --hash-table-size 6000000 \
            --tmpdir=/moto/eaton/projects/macaques/tmp/ \
            /moto/eaton/projects/macaques/mapped/'+i+'.NC_018152.2.sorted.bam \
            /moto/eaton/projects/macaques/mapped/'+i+'.NC_018152.2.mark.bam'
    os.system(cmd)

In [ ]:
%%bash
for i in NC_018152.2 NC_018153.2 NC_018154.2; do
    samtools view \
        /moto/eaton/projects/macaques/mulattanorthern/filtercall/mulattanorthern.SRR4454026.raw.minimap2.sorted.bam $i -b > \
        /moto/eaton/projects/macaques/mulattanorthern/filtercall/mulattanorthern.$i.bam


In [ ]:
%%bash
for i in NC_018152.2 NC_018153.2 NC_018154.2 NC_018155.2 NC_018156.2 NC_018157.2 NC_018158.2 NC_018159.2 NC_018160.2 NC_018161.2 NC_018162.2 NC_018163.2 NC_018164.2 NC_018165.2 NC_018166.2 NC_018167.2 NC_018168.2 NC_018169.2 NC_018170.2 NC_018171.2 NC_018172.2 NC_020006.2; do
    sambamba sort -t 24 -p --tmpdir=/moto/eaton/projects/macaques/mulattanorthern/filtercall/ \
        /moto/eaton/projects/macaques/mulattanorthern/filtercall/mulattanorthern.$i.bam
done

In [ ]:
%%bash
for i in NC_018152.2 NC_018153.2 NC_018154.2 NC_018155.2 NC_018156.2 NC_018157.2 NC_018158.2 NC_018159.2 NC_018160.2 NC_018161.2 NC_018162.2 NC_018163.2 NC_018164.2 NC_018165.2 NC_018166.2 NC_018167.2 NC_018168.2 NC_018169.2 NC_018170.2 NC_018171.2 NC_018172.2 NC_020006.2; do
    sambamba markdup -t 24 -p --tmpdir=/moto/eaton/projects/macaques/mulattanorthern/filtercall/ \
        /moto/eaton/projects/macaques/mulattanorthern/filtercall/mulattanorthern.$i.sorted.bam \
        /moto/eaton/projects/macaques/mulattanorthern/filtercall/mulattanorthern.$i.ready.bam
done

In [2]:
!mkdir /moto/eaton/projects/macaques/MAPPEDANDSORTED

In [3]:
##moving our mapped and sorted reads, separated by chromosome/mitochondria, to a staging directory
!mv /moto/eaton/projects/macaques/mulattanorthern/filtercall/*.ready.* \
    /moto/eaton/projects/macaques/MAPPEDANDSORTED

### B) Mapping and sorting southern, low altitude _Macaca mulatta_:

In [4]:
!mkdir /moto/eaton/projects/macaques/mulattasouthernlow/filtercall

In [ ]:
%time !minimap2 -ax sr -t 24 /moto/eaton/projects/macaques/refpapio/refpapio.fa \
    /moto/eaton/projects/macaques/TRIM/mulattasouthernlowSRR4454020_1.fastq.gz \
    /moto/eaton/projects/macaques/TRIM/mulattasouthernlowSRR4454020_2.fastq.gz \
    | samtools view -b -> /moto/eaton/projects/macaques/mulattasouthernlow/filtercall/mulattasouthernlow.SRR4454020.raw.minimap2.bam

In [6]:
%time !sambamba sort -t 24 -p --tmpdir=/moto/eaton/projects/macaques/mulattasouthernlow/filtercall/ \
    /moto/eaton/projects/macaques/mulattasouthernlow/filtercall/mulattasouthernlow.SRR4454020.raw.minimap2.bam


sambamba 0.6.8 by Artem Tarasov and Pjotr Prins (C) 2012-2018
    LDC 1.11.0 / DMD v2.081.2 / LLVM6.0.1 / bootstrap LDC - the LLVM D compiler (0.17.6git-0156298)

Writing sorted chunks to temporary directory...
[==============================================================================]
Merging sorted chunks...
[==============================================================================]
CPU times: user 14.1 s, sys: 869 ms, total: 14.9 s
Wall time: 5min 28s


In [10]:
%%bash
for i in NC_018152.2 NC_018153.2 NC_018154.2 NC_018155.2 NC_018156.2 NC_018157.2 NC_018158.2 NC_018159.2 NC_018160.2 NC_018161.2 NC_018162.2 NC_018163.2 NC_018164.2 NC_018165.2 NC_018166.2 NC_018167.2 NC_018168.2 NC_018169.2 NC_018170.2 NC_018171.2 NC_018172.2 NC_020006.2; do
    samtools view \
        /moto/eaton/projects/macaques/mulattasouthernlow/filtercall/mulattasouthernlow.SRR4454020.raw.minimap2.sorted.bam $i -b > \
        /moto/eaton/projects/macaques/mulattasouthernlow/filtercall/mulattasouthernlow.$i.bam
done

In [ ]:
%%bash
for i in NC_018152.2 NC_018153.2 NC_018154.2 NC_018155.2 NC_018156.2 NC_018157.2 NC_018158.2 NC_018159.2 NC_018160.2 NC_018161.2 NC_018162.2 NC_018163.2 NC_018164.2 NC_018165.2 NC_018166.2 NC_018167.2 NC_018168.2 NC_018169.2 NC_018170.2 NC_018171.2 NC_018172.2 NC_020006.2; do
    sambamba sort -t 24 -p --tmpdir=/moto/eaton/projects/macaques/mulattasouthernlow/filtercall/ \
        /moto/eaton/projects/macaques/mulattasouthernlow/filtercall/mulattasouthernlow.$i.bam
done

In [ ]:
%%bash
for i in NC_018152.2 NC_018153.2 NC_018154.2 NC_018155.2 NC_018156.2 NC_018157.2 NC_018158.2 NC_018159.2 NC_018160.2 NC_018161.2 NC_018162.2 NC_018163.2 NC_018164.2 NC_018165.2 NC_018166.2 NC_018167.2 NC_018168.2 NC_018169.2 NC_018170.2 NC_018171.2 NC_018172.2 NC_020006.2; do
    sambamba markdup -t 24 -p --tmpdir=/moto/eaton/projects/macaques/mulattasouthernlow/filtercall/ \
        /moto/eaton/projects/macaques/mulattasouthernlow/filtercall/mulattasouthernlow.$i.sorted.bam \
        /moto/eaton/projects/macaques/mulattasouthernlow/filtercall/mulattasouthernlow.$i.ready.bam
done

In [13]:
!mv /moto/eaton/projects/macaques/mulattasouthernlow/filtercall/*.ready.* \
    /moto/eaton/projects/macaques/MAPPEDANDSORTED

### C) Mapping and sorting southern, high altitude _Macaca mulatta_:

In [14]:
!mkdir /moto/eaton/projects/macaques/mulattasouthernhigh/filtercall

In [ ]:
%time !minimap2 -ax sr -t 24 /moto/eaton/projects/macaques/refpapio/refpapio.fa \
    /moto/eaton/projects/macaques/TRIM/mulattasouthernhighSRR4453966_1.fastq.gz \
    /moto/eaton/projects/macaques/TRIM/mulattasouthernhighSRR4453966_2.fastq.gz \
    | samtools view -b -> /moto/eaton/projects/macaques/mulattasouthernhigh/filtercall/mulattasouthernhigh.SRR4453966.raw.minimap2.bam

In [16]:
%time !sambamba sort -t 24 -p --tmpdir=/moto/eaton/projects/macaques/mulattasouthernhigh/filtercall/ \
    /moto/eaton/projects/macaques/mulattasouthernhigh/filtercall/mulattasouthernhigh.SRR4453966.raw.minimap2.bam


sambamba 0.6.8 by Artem Tarasov and Pjotr Prins (C) 2012-2018
    LDC 1.11.0 / DMD v2.081.2 / LLVM6.0.1 / bootstrap LDC - the LLVM D compiler (0.17.6git-0156298)

Writing sorted chunks to temporary directory...
[==============================================================================]=========================>                                                 ]
Merging sorted chunks...
[==============================================================================]=============================================>             ]
CPU times: user 16.7 s, sys: 1.01 s, total: 17.7 s
Wall time: 6min 16s


In [17]:
%%bash
for i in NC_018152.2 NC_018153.2 NC_018154.2 NC_018155.2 NC_018156.2 NC_018157.2 NC_018158.2 NC_018159.2 NC_018160.2 NC_018161.2 NC_018162.2 NC_018163.2 NC_018164.2 NC_018165.2 NC_018166.2 NC_018167.2 NC_018168.2 NC_018169.2 NC_018170.2 NC_018171.2 NC_018172.2 NC_020006.2; do
    samtools view \
        /moto/eaton/projects/macaques/mulattasouthernhigh/filtercall/mulattasouthernhigh.SRR4453966.raw.minimap2.sorted.bam $i -b > \
        /moto/eaton/projects/macaques/mulattasouthernhigh/filtercall/mulattasouthernhigh.$i.bam
done

In [ ]:
%%bash
for i in NC_018152.2 NC_018153.2 NC_018154.2 NC_018155.2 NC_018156.2 NC_018157.2 NC_018158.2 NC_018159.2 NC_018160.2 NC_018161.2 NC_018162.2 NC_018163.2 NC_018164.2 NC_018165.2 NC_018166.2 NC_018167.2 NC_018168.2 NC_018169.2 NC_018170.2 NC_018171.2 NC_018172.2 NC_020006.2; do
    sambamba sort -t 24 -p --tmpdir=/moto/eaton/projects/macaques/mulattasouthernhigh/filtercall/ \
        /moto/eaton/projects/macaques/mulattasouthernhigh/filtercall/mulattasouthernhigh.$i.bam
done

In [19]:
%%bash
for i in NC_018152.2 NC_018153.2 NC_018154.2 NC_018155.2 NC_018156.2 NC_018157.2 NC_018158.2 NC_018159.2 NC_018160.2 NC_018161.2 NC_018162.2 NC_018163.2 NC_018164.2 NC_018165.2 NC_018166.2 NC_018167.2 NC_018168.2 NC_018169.2 NC_018170.2 NC_018171.2 NC_018172.2 NC_020006.2; do
    sambamba markdup -t 24 -p --tmpdir=/moto/eaton/projects/macaques/mulattasouthernhigh/filtercall/ \
        /moto/eaton/projects/macaques/mulattasouthernhigh/filtercall/mulattasouthernhigh.$i.sorted.bam \
        /moto/eaton/projects/macaques/mulattasouthernhigh/filtercall/mulattasouthernhigh.$i.ready.bam
done

Process is interrupted.


In [ ]:
!mv /moto/eaton/projects/macaques/mulattasouthernhigh/filtercall/*.ready.* \
    /moto/eaton/projects/macaques/MAPPEDANDSORTED

#### SAM to BAM conversion and sorting reads:

In [6]:
%time !samtools view -S -b results.sam > sample.bam ##simple conversion to bam appx 21 min on a 12 thread desktop w/ 16gb ram, not bad

CPU times: user 23.8 s, sys: 3.48 s, total: 27.3 s
Wall time: 21min 13s


In [7]:
%time !samtools sort sample.bam -o sample.sorted.bam ##sorting bam file into genome order ~26mins

[bam_sort_core] merging from 53 files and 1 in-memory blocks...
CPU times: user 28.7 s, sys: 3.85 s, total: 32.6 s
Wall time: 25min 41s


In [14]:
%time !samtools index sample.sorted.bam ##of course this will all be piped together...

CPU times: user 2.36 s, sys: 363 ms, total: 2.73 s
Wall time: 1min 57s


In [1]:
%time !samtools view sample.sorted.bam | head -n 1 ##We see that instead of giving chromosomes logical names like Chr1, Chr2, etc., the reference genome has strange names for chromosomes (NC_027893.1, etc)...

SRR445694~125200.sra.858593	99	NC_027893.1	1	60	5S95M	=	294	393	AAGGCCATGGAAACAAGGAAAGTCTGAAAAACTCACAGTTTAGGAACCTAAAGAGACTTGACTACTAAATGGAATATATCTTGGGATCCTGGAAAAGAAA	CCCFFFFFHHHHHIIIIIIIIIHIIIIIIIIIIIIIIIHIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIHFHHHHFFFFFFDDDDCCCDBCCDDBDD	AS:i:950	NM:i:0	NH:i:0	XI:f:1	X0:i:0	XE:i:28	XR:i:95	MD:Z:95
samtools view: writing to standard output failed: Broken pipe
samtools view: error closing standard output: -1
CPU times: user 5.74 ms, sys: 7.92 ms, total: 13.7 ms
Wall time: 224 ms


In [3]:
%time !samtools view -h -b sample.sorted.bam NC_027893.1 > chr1.bam ##Which makes splitting files up for chromosome-level analyses a bit more annoying but not too bad...I'll make a bash script

CPU times: user 1.19 s, sys: 147 ms, total: 1.34 s
Wall time: 58.6 s


Pipe from NGM to samtools with an output of a sorted bam file:

In [ ]:
!ngm -r ./reference-genome/Mmul8.fna.gz -1 out.R1.fq.gz -2 out.R2.fq.gz | samtools view -S -b | samtools sort -o sample.sorted.bam

#### Variant calling:

In [ ]:
!freebayes -f ./reference-genome/Mmul8.fna.gz sample.sorted.bam >wholegenome.vcf ##example code for variant calling on entire genome. Can be split by chromosome/region using -r 